In [1]:
!pip uninstall torch -y
!pip uninstall torch -y
! pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html --use-feature=2020-resolver
import torch
print(torch.__version__)

Found existing installation: torch 1.5.1
Uninstalling torch-1.5.1:
  Successfully uninstalled torch-1.5.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 7.5 kB/s  eta 0:00:01  |▏                               | 5.2 MB 2.2 MB/s eta 0:05:43     |███▉                            | 90.5 MB 26.4 MB/s eta 0:00:26     |████▉                           | 114.7 MB 37.2 MB/s eta 0:00:18     |████▉                           | 116.2 MB 37.2 MB/s eta 0:00:18     |█████                           | 119.4 MB 37.2 MB/s eta 0:00:18     |██████████████▏                 | 338.3 MB 371 kB/s eta 0:19:05     |██████████████▎                 | 339.8 MB 371 kB/s eta 0:19:01     |██████████████▌                 | 344.9 MB 371 kB/s eta 0:18:47     |████████████████████████████▍   | 677.5 MB 14.7 MB/s eta 0:00:06     |█████████████████████████████▏  | 694.7 MB 17.0 MB/s eta 0:00:05     |█████████████████████████████▍  | 699.9 MB 17.0 MB/s eta 0:

In [ ]:
# shift command P -> confirm restart kernel
import torch
print(torch.__version__)

In [2]:
!pip install attrdict
!pip install pytorch-lightning==0.9.0
!pip install omegaconf

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 408 kB 868 kB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 10.9 MB/s eta 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
tensorflow 2.3.0 requires tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgra

In [7]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
from argparse import Namespace
from utils.utils import get_batch_k
import numpy as np
from omegaconf import DictConfig
import torch.nn as nn
from utils.utils import l2_loss, GANLoss, cal_ade, cal_fde, crashIntoWall
from models.model_main import get_model
import torch
from data.collect_data import TrajectoryDataset, seq_collate
from data.collect_data import TrajectoryDataset, seq_collate
from torch.utils.data import DataLoader
from utils.utils import RAdam, re_im, visualize_traj_probabilities

import os
import itertools
import psutil

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer

from pytorch_lightning import Trainer, seed_everything
import os
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_lightning.callbacks import ModelCheckpoint
import logging
import torch

## Configs

In [8]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': "/kaggle/input/lyft-motion-prediction-autonomous-vehicles",
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "model_resnet34_output",
        'lr': 1e-3,
        'train': True,
        'predict': False
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    'val_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'max_num_steps': 101,
        'checkpoint_every_n_steps': 20,
    }
}

## Load the train and test data

In [9]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

In [10]:
# ===== INIT TRAIN DATASET============================================================
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
# train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
#                              num_workers=train_cfg["num_workers"])
print("==================================TRAIN DATA==================================")
print(train_dataset)

==================================TRAIN DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [ ]:
#====== INIT TEST DATASET=============================================================
test_cfg = cfg["test_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
# test_dataloader = DataLoader(test_dataset,shuffle=test_cfg["shuffle"],batch_size=test_cfg["batch_size"],
#                              num_workers=test_cfg["num_workers"])
print("==================================TEST DATA==================================")
print(test_dataset)

In [11]:
from tqdm import tqdm_notebook

## Run model

In [20]:
import yaml
with open(r'/kaggle/input/config/config.yaml') as file:
    cfg = yaml.load(file, Loader=yaml.FullLoader)

In [21]:
tb_logger = TensorBoardLogger(".", name = "{}".format(cfg['dataset_name']))

if cfg['make_checkpoint']:
    checkpoint_location = os.path.join(tb_logger.log_dir,"checkpoints")
    logger.info("Checkpoint Location: {}".format(checkpoint_location))
    checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(tb_logger.log_dir,"checkpoints", 'checkpoint'),
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

Missing logger folder: ./lyft


In [22]:
model = TrajPredictor(cfg)

GoalGAN(
  (encoder): MotionEncoder(
    (spatial_embedding): Linear(in_features=2, out_features=8, bias=True)
    (encoder): LSTM(8, 16)
  )
  (goalmodule): GoalGlobal(
    (CNN): CNN(
      (encoder): Sequential(
        (ConvBlock_1): Conv_Blocks(
          (Block): Sequential(
            (Conv_1): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (BN_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (NonLin_1): ReLU()
            (Pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=False)
            (Drop): Dropout2d(p=0.3, inplace=False)
          )
        )
        (ConvBlock_2): Conv_Blocks(
          (Block): Sequential(
            (Conv_1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (BN_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (NonLin_1): ReLU()
            (Pool): MaxPool2d(ker

In [23]:
data_dict = {}
data_dict["train"] = TrajectoryDataset( data=train_dataset, phase = "train" )
data_dict["test"] = TrajectoryDataset( data=train_dataset, phase = "test" )

In [24]:
model.setupData(data_dict)

## Pretraing for estimating goal

In [25]:
cfg['trainer']['gpus']=None

In [26]:
del train_dataset
import gc
gc.collect()

72

In [27]:
if cfg['pretrain']:
    pretrain_func(generator=model.generator,
                  train_dset=model.train_dset,
                  val_dset=model.val_dset,
                  cfg=cfg,
                  logger=tb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | generator | GoalGAN | 21 K  


Saving latest checkpoint..


In [ ]:
if cfg['batch_size_scheduler']:
    callbacks = [BatchSizeScheduler(	bs = cfg['batch_size_scheduler'],
                                        max_bs = cfg['batch_size'] )]
else:
    callbacks = False

trainer = Trainer(logger=tb_logger,
                  callbacks=callbacks,
                  checkpoint_callback=checkpoint_callback,
                  num_sanity_val_steps=0,
                  progress_bar_refresh_rate=10,
                  **cfg['trainer'])
trainer.fit(model)

logger.info("Loading best model")
trainer.test(ckpt_path='best')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                 | Params
-------------------------------------------------------
0 | generator     | GoalGAN              | 21 K  
1 | discriminator | GoalGANDiscriminator | 5 K   



Epoch 00000: val_loss reached 2.78461 (best 2.78461), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_0.ckpt as top 1



Epoch 00001: val_loss reached 1.98115 (best 1.98115), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_1.ckpt as top 1



Epoch 00002: val_loss reached 1.84190 (best 1.84190), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_2.ckpt as top 1



Epoch 00003: val_loss reached 1.77203 (best 1.77203), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_3.ckpt as top 1


...


Epoch 00042: val_loss  was not in top 1



Epoch 00043: val_loss  was not in top 1



Epoch 00044: val_loss  was not in top 1



Epoch 00045: val_loss reached 1.39094 (best 1.39094), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_45.ckpt as top 1



Epoch 00046: val_loss reached 1.38929 (best 1.38929), saving model to /kaggle/working/lyft/version_0/checkpoints/_ckpt_epoch_46.ckpt as top 1
